In [2]:
import pandas as pd

import pickle

import numpy as np

import re
import string

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style('white')

from scipy.spatial.distance import cosine
from scipy.spatial.distance import cdist

import operator

In [21]:
with open ('../models/05_kojak_demo.pkl', 'rb') as picklefile:
    df_loc, df_doc_topics, nmf_topics = pickle.load(picklefile)

In [30]:
neighborhoods = df_loc['neighborhood'].apply(lambda u: u.encode('utf-8')).tolist()
cities = df_loc['city'].tolist()
topics = df_loc['topic'].tolist()

zipped = zip(neighborhoods, cities, topics)

In [23]:
test_list = [10.0, 10.0, 6.0, 10.0, \
       10.0, 7.0, 10.0, 0.0, \
       0.0, 10.0, 9.0, 10.0, \
       10.0, 10.0, 10.0, 10.0]

In [24]:
test_array = np.array(test_list)

In [25]:
nmf_topics[test_list.index(max(test_list))]

'1_taco'

In [26]:
test_distribution = test_array/sum(test_array)

In [27]:
sum(test_distribution)

1.0

In [31]:
def find_comps(test_distribution, num_comps=5):
    dists = {}
    for i in range(len(df_doc_topics)):
        dists[i] = cosine(df_doc_topics.ix[i].T, np.array(test_distribution))
    max_keys = dict(sorted(dists.iteritems(), key=operator.itemgetter(1), reverse=True)[:num_comps]).keys()
    print max_keys
    for key in max_keys:
        print zipped[key]

In [32]:
find_comps(test_distribution)

[852, 225, 715, 524, 201]
('West Loop Gate', 'chicago', '9_nightlife')
('East Pilsen', 'chicago', '8_art')
('SoHo', 'newyorkcity', '9_nightlife')
('Near North', 'chicago', '9_nightlife')
('Downtown', 'sanfrancisco', '9_nightlife')
